# Introduction to GPU Programming Summer School
## Tutorial 1. Using CUDA accelerated libraries

### Step 1 : Sanity check of Python environment

All being well, you've reading this text inside a Jupyter notebook environment running on an SCRTP machine equipped with a CUDA-compatible GPU card and have launched the notebook from within an environment configured to support GPU computing. If in doubt, check the connecting instructions.

https://warwick.ac.uk/fac/sci/maths/research/events/2017-18/nonsymposium/gpu/connecting


Let's run some simpple checks to make sure you can execute code on a GPU from within this notebook. For today we'll be working with the Python interface to CUDA provided via the numba package.

In [5]:
from numba import cuda   # Import Numba

if cuda.is_available():  # Test if CUDA is available on this system
    cuda.detect()        # and report on the hardware available
else:
    print("There doesn't appear to be a CUDA capable device in this system")

Found 2 CUDA devices
id 0           b'Tesla K20c'                              [SUPPORTED]
                      compute capability: 3.5
                           pci device id: 0
                              pci bus id: 4
id 1              b'NVS 310'                              [SUPPORTED]
                      compute capability: 2.1
                           pci device id: 0
                              pci bus id: 3
Summary:
	2/2 devices are supported
